In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [734 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [731 kB]
Fetched 1,802 kB in 1s (1,414 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"", timestampFormat="yyyy/MM/dd HH:mm:ss")

# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, avg, round

four_bedroom_df = df.filter(df["bedrooms"] == 4)

# Extract the year from the 'date' column
four_bedroom_df = four_bedroom_df.withColumn("year", year(four_bedroom_df["date"]))

# Group the data by year and calculate the average price
average_price_by_year_df = four_bedroom_df.groupBy("year").agg(round(avg("price"), 2).alias("average_price")).show()

+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
bedroom_df = df.filter((df["bedrooms"] == 3) & (df["bathrooms"] == 3))

# Extract the year from the 'date' column
bedroom_df = bedroom_df.withColumn("year", year(bedroom_df["date"]))

# Group the data by year and calculate the average price
average_price_by_year_df = bedroom_df.groupBy("year").agg(round(avg("price"), 2).alias("average_price")).show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    292725.69|
|2019|    287287.82|
|2020|    294204.16|
|2021|    294211.46|
+----+-------------+



In [7]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
square_feet_df = df.filter(
    (df["bedrooms"] == 3) &
    (df["bathrooms"] == 3) &
    (df["floors"] == 2) &
     (df["sqft_living"] >= 2000))

# Extract the year from the 'date' column
square_feet_df = square_feet_df.withColumn("year", year(square_feet_df["date"]))

# Group the data by year and calculate the average price
average_price_by_sqft = square_feet_df.groupBy("year").agg(round(avg("price"), 2).alias("average_price")).show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    290242.99|
|2019|    289859.14|
|2020|    292289.09|
|2021|    296330.96|
+----+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.200241088867188e-05 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
df = spark.table("home_sales")
df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [27]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_cached = time.time()
query_result_cached = spark.sql("SELECT view, price FROM home_sales WHERE price >= 350000 ORDER BY price DESC").show()
runtime_cached = time.time() - start_time_cached
print(f"Runtime with cached data: {runtime_cached} seconds")
spark.catalog.uncacheTable("home_sales")
start_time_uncached = time.time()
query_result_uncached = spark.sql("SELECT * FROM home_sales WHERE price >= 350000")
print(f"Runtime with uncached data : %s seconds" % (time.time() - start_time))


+----+-------+
|view|  price|
+----+-------+
|  84|1499741|
|  75|1498959|
|  82|1498689|
|  98|1498506|
|  89|1497982|
|  95|1497910|
|  94|1496919|
|  91|1495146|
|  93|1494589|
|  82|1492438|
|  90|1492359|
|  87|1492151|
|  82|1492045|
|  83|1491885|
|  87|1491580|
|  86|1489964|
|  91|1489692|
| 100|1488977|
|  81|1488343|
|  90|1488101|
+----+-------+
only showing top 20 rows

Runtime with cached data: 0.3418138027191162 seconds
Runtime with uncached data : 1977.8139414787292 seconds


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [32]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet("home_sales")

In [33]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('home_sales')

In [34]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time_cached = time.time()
query_result_cached = spark.sql("SELECT view, price FROM home_sales WHERE price >= 350000 ORDER BY price DESC").show()
runtime_cached = time.time() - start_time_cached
print(f"Runtime with cached data: {runtime_cached} seconds")
spark.catalog.uncacheTable("home_sales")
start_time_uncached = time.time()
query_result_uncached = spark.sql("SELECT * FROM home_sales WHERE price >= 350000")
print(f"Runtime with uncached data : %s seconds" % (time.time() - start_time))

+----+-------+
|view|  price|
+----+-------+
|  84|1499741|
|  75|1498959|
|  82|1498689|
|  98|1498506|
|  89|1497982|
|  95|1497910|
|  94|1496919|
|  91|1495146|
|  93|1494589|
|  82|1492438|
|  90|1492359|
|  87|1492151|
|  82|1492045|
|  83|1491885|
|  87|1491580|
|  86|1489964|
|  91|1489692|
| 100|1488977|
|  81|1488343|
|  90|1488101|
+----+-------+
only showing top 20 rows

Runtime with cached data: 0.7842533588409424 seconds
Runtime with uncached data : 4365.437618970871 seconds


In [35]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [36]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is 'home_sales' cached? {is_cached}")


Is 'home_sales' cached? False
